In [1]:
!pip install -e ..

Obtaining file:///export/home/figs/vdesouza/work/Holography-1022/astrohack
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for astrohack (pyproject.toml) ... done
  Created wheel for astrohack: filename=astrohack-0.0.1-py3-none-any.whl size=1061 sha256=59f59d1e6272b2f472c5ed799d07c7c8fb2fa0681314ade2716b2f218436951c
  Stored in directory: /tmp/pip-ephem-wheel-cache-a_5ab0l_/wheels/93/bb/ed/46bdf801cce81c3aea69f1f4af83e386e0f89149047ab78ae0
Successfully built astrohack
  Attempting uninstall: astrohack
    Found existing installation: astrohack 0.0.1
    Uninstalling astrohack-0.0.1:
      Successfully uninstalled astrohack-0.0.1


In [2]:
import gdown
import shutil
from astrohack.panel import panel
from astrohack._utils._io import _aips_holog_to_xds
import os
from dask.distributed import Client, LocalCluster

In [3]:
# Download VLA Holography with known panel deviations
datafolder = "./paneldata/"
os.makedirs(name=datafolder, exist_ok=True)
panelzip = datafolder+"panel.zip"
if not os.path.exists(panelzip):
    url = "https://drive.google.com/u/1/uc?id=10fXyut_UHPUjIuaaEy6-m6wcycZHit2v&export=download"
    gdown.download(url, panelzip)
shutil.unpack_archive(filename=panelzip, extract_dir=datafolder)

In [4]:
# Create .zarr data set from AIPS data
holog_image = datafolder+'input.zarr'
input_data_xds = _aips_holog_to_xds(datafolder+'amp.fits', datafolder+'dev.fits')
input_data_xds.to_zarr(holog_image, mode='w')

In [5]:
%%time
# Execute Panel on a single VLA dataset
panel(holog_image, 'vla_data',
      aipsdata=True, 
      telescope='vla', 
      basename='antenna25', 
      unit='mils', 
      optimize=0)

[2023-03-22 17:24:12.735160 System Info]: inrms:0.482952; ourms:0.221199
CPU times: user 4.44 s, sys: 267 ms, total: 4.71 s
Wall time: 4.41 s


In [6]:
%%time
# Execute Panel on a single VLA dataset
panel(holog_image, 'vla_data',
      aipsdata=True, 
      telescope='vla', 
      basename='antenna25', 
      unit='mils', 
      optimize=1)

[2023-03-22 17:24:15.011626 System Info]: inrms:0.482952; ourms:0.214440
CPU times: user 2.32 s, sys: 276 ms, total: 2.59 s
Wall time: 2.27 s


In [7]:
%%time
# Execute Panel on a single VLA dataset
panel(holog_image, 'vla_data',
      aipsdata=True, 
      telescope='vla', 
      basename='antenna25', 
      unit='mils', 
      optimize=2)

[2023-03-22 17:24:15.611548 System Info]: inrms:0.482952; ourms:0.216367
CPU times: user 646 ms, sys: 272 ms, total: 918 ms
Wall time: 586 ms


In [8]:
%%time
holog_image = 'hack_file'
panel_output = 'hack_file'
panel(holog_image, panel_output,
      parallel=False,
      optimize=0)

[2023-03-22 17:24:15.659923 System Info]: Processing ddi_0 for ant_3
[2023-03-22 17:24:28.219197 System Info]: inrms:0.042120; ourms:0.011616
CPU times: user 12.7 s, sys: 547 ms, total: 13.2 s
Wall time: 12.6 s


In [9]:
%%time
holog_image = 'hack_file'
panel_output = 'hack_file'
panel(holog_image, panel_output,
      parallel=False,
      optimize=1)

[2023-03-22 17:24:28.295441 System Info]: Processing ddi_0 for ant_3
[2023-03-22 17:24:35.729927 System Info]: inrms:0.042120; ourms:0.010982
CPU times: user 7.58 s, sys: 487 ms, total: 8.07 s
Wall time: 7.5 s


In [10]:
%%time
holog_image = 'hack_file'
panel_output = 'hack_file'
panel(holog_image, panel_output,
      parallel=False,
      optimize=2)

[2023-03-22 17:24:35.804748 System Info]: Processing ddi_0 for ant_3
[2023-03-22 17:24:37.718623 System Info]: inrms:0.042120; ourms:0.010975
CPU times: user 2.07 s, sys: 497 ms, total: 2.56 s
Wall time: 1.98 s
